In [ ]:
import pandas as pd
import numpy as np
import re
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import geopandas
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
from typing import Tuple, Dict, List
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import rpy2.robjects.packages as rpackages
gamlss = rpackages.importr("gamlss")
gamlss_dist = rpackages.importr("gamlss.dist")
base = rpackages.importr("base")
import xgboost as xgb

In [ ]:
carpeta_path = r'C:\Users\William\Desktop\el algoritmo es correcto'
archivo_train = r'train_data.csv'
path_train = os.path.join(carpeta_path, archivo_train)

In [ ]:
train = pd.read_csv(
    path_train, 
    dtype={
        "tiempodeconstruido": str,
        "vista": str,
        "tipoinmueble": str,
        "tiponegocio": str,
        'valorventa': 'int64',
    }
)
train['data'] = 'train'

In [ ]:
carpeta_path = r'C:\Users\William\Desktop\el algoritmo es correcto'
archivo_test = r'test_data.csv'
path_test = os.path.join(carpeta_path, archivo_test)

In [ ]:
test = pd.read_csv(
    path_test, 
    dtype={
        "tiempodeconstruido": str,
        "vista": str,
        "tipoinmueble": str,
        "tiponegocio": str,
        'valorventa': 'int64'
    }
)
test['data'] = 'test'

In [ ]:
prueba = pd.concat([train, test], ignore_index=True)

In [ ]:
prueba = prueba[
    (prueba['porteria'].isna()) &
    (prueba['conjuntocerrado']==0) &
    (prueba['tipoinmueble']=='Casa') &
    (prueba['ascensor'].isna()) &
    ((prueba['gimnasio'].isna()) | (prueba['gimnasio']==0)) &
    (prueba['piscina'].isna()) &
    ((prueba['area']<1500) & (prueba['area']>35)) &
    ((prueba['piso']<5) | (prueba['piso'].isna())) &
    ((prueba['habitaciones']<15) & (prueba['habitaciones']>0))
]

In [ ]:
columnas_no_usar = ['ascensor', 'tipoinmueble', 'porteria', 'conjuntocerrado', 'gimnasio', 'piscina', 'valoradministracion','tiponegocio']
prueba = prueba.drop(columns = columnas_no_usar)

In [ ]:
condlist = [
    (prueba['banos']<=1) | (prueba['banos'].isna()), 
    (prueba['banos']==2), 
    (prueba['banos'].isin([3,4,5])), 
    (prueba['banos'].isin([6,7,8,9])),
    (prueba['banos']>=10)]
choicelist = ['1', '2', '3-5', '6-9', '10+']
prueba['banos modificado'] = np.select(condlist=condlist, choicelist=choicelist)

In [ ]:
condlist = [
    (prueba['garajes'].isna()),
    (prueba['garajes']<=1), 
    (prueba['garajes']>1) & (prueba['garajes']<=3), 
    (prueba['garajes']>3)]
choicelist = ['N.N', '0-1', '2-3', '4+']
prueba['garajes modificado'] = np.select(condlist=condlist, choicelist=choicelist)

In [ ]:
condlist = [(prueba['piso'].isna()), (prueba['piso']>0)]
choicelist = ['No define', prueba['piso'].astype(str)]
prueba['piso modificado'] = np.select(condlist=condlist, choicelist=choicelist)

In [ ]:
condlist = [(prueba['vista'].isna()), (prueba['vista'].notna())]
choicelist = ['No define', prueba['vista']]
prueba['vista modificado'] = np.select(condlist=condlist, choicelist=choicelist)

In [ ]:
condlist = [(prueba['zonalavanderia'].isna()), (prueba['zonalavanderia'].notna())]
choicelist = [0, 1]
prueba['zonalavanderia modificado'] = np.select(condlist=condlist, choicelist=choicelist) #sale zonalavanderia del modelo

In [ ]:
condlist = [(prueba['vigilancia'].isna()) | (prueba['vigilancia']==0), (prueba['vigilancia']==1)]
choicelist = ['No define-0', prueba['vigilancia'].astype(str)]
prueba['vigilancia modificado'] = np.select(condlist=condlist, choicelist=choicelist)

In [ ]:
condlist = [(prueba['deposito'].isna()), (prueba['deposito'].notna())]
choicelist = [0, prueba['deposito']]
prueba['deposito modificado'] = np.select(condlist=condlist, choicelist=choicelist)

In [ ]:
prueba['tiempodeconstruido modificado'] = np.where(
    prueba['tiempodeconstruido'].isna(),
    'No define',
    prueba['tiempodeconstruido']
)

In [ ]:
columnas_valor0 = [
    'balcon', 'banoservicio', 'cuartoservicio', 'estudio', 'halldealcobas', 
    'remodelado', 'parqueaderovisitantes', 'saloncomunal','terraza']
prueba[columnas_valor0] = prueba[columnas_valor0].fillna(0)

In [ ]:
prueba['y'] = (prueba['valorventa']/prueba['area'])

In [ ]:
prueba = prueba[
    (prueba['y']<prueba['y'].quantile(0.95).astype('int64')) &
    (prueba['y']>prueba['y'].quantile(0.05).astype('int64'))]

In [ ]:
print(min(prueba['y']))
print(max(prueba['y']))

##  Banos

In [ ]:
sns.boxplot(y='y', x='banos modificado', data=prueba)
plt.show()

## Garaje

In [ ]:
sns.boxplot(y='y', x='garajes modificado', data=prueba)
plt.show()

## Piso

In [ ]:
sns.boxplot(y='y', x='piso modificado', data=prueba, order=['No define', '1.0', '2.0', '3.0', '4.0'])
plt.show()

## Vista

In [ ]:
sns.violinplot(y='y', x='vista modificado', data=prueba)
plt.show()

In [ ]:
sns.violinplot(y='y', x='vista modificado', data=prueba)
plt.show()

## Zona de lavanderia

In [ ]:
sns.violinplot(y='y', x='zonalavanderia modificado', data=prueba)
plt.show()

## Vigilancia

In [ ]:
sns.boxplot(y='y', x='vigilancia modificado', data=prueba)
plt.show()

## Deposito

In [ ]:
sns.boxplot(y='y', x='deposito modificado', data=prueba)
plt.show()

## Tiempo de construido

In [ ]:
sns.boxplot(x='y', y='tiempodeconstruido modificado', data=prueba, orient="h", 
    order=[
        "No define", "Remodelado", "Menos de 1 año", "1 a 8 años", "Entre 0 y 5 años", 
        "Entre 5 y 10 años", "9 a 15 años", "Entre 10 y 20 años", "Más de 20 años", 
        "16 a 30 años", "Más de 30 años"])
plt.show()

In [ ]:
sns.boxplot(y='y', x='balcon', data=prueba)
plt.show()

In [ ]:
sns.boxplot(y='y', x='banoservicio', data=prueba)
plt.show()

In [ ]:
sns.boxplot(y='y', x='cuartoservicio', data=prueba)
plt.show()

In [ ]:
sns.boxplot(y='y', x='estudio', data=prueba)
plt.show()

In [ ]:
sns.violinplot(y='y', x='halldealcobas', data=prueba)
plt.show()

In [ ]:
sns.violinplot(y='y', x='remodelado', data=prueba)
plt.show()

In [ ]:
sns.violinplot(y='y', x='parqueaderovisitantes', data=prueba)
plt.show()

In [ ]:
sns.boxplot(y='y', x='saloncomunal', data=prueba)
plt.show()

In [ ]:
sns.boxplot(y='y', x='terraza', data=prueba)
plt.show()

In [ ]:
sns.pairplot(prueba[['y', "habitaciones", "area", "valorventa", "garajes modificado"]].sample(5000), hue='garajes modificado')
plt.show()

## Union de datos Abierto Bogotá

In [ ]:
localidades_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\localidades.json'
localidades = geopandas.read_file(localidades_path, crs={'init':'epsg:4686'})

In [ ]:
localidades = localidades[['LocNombre', 'geometry']]

In [ ]:
estratificacion_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\manzana_estratificacion.json'
estratificacion = geopandas.read_file(estratificacion_path, crs={'init':'epsg:4686'})

In [ ]:
estratificacion = estratificacion[['ESTRATO', 'geometry']]

In [ ]:
casas = geopandas.GeoDataFrame(
    prueba, geometry=geopandas.points_from_xy(prueba.longitud, prueba.latitud), crs={'init': 'epsg:4326'})

In [ ]:
avaluo_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\avaluo'
avaluo = geopandas.read_file(avaluo_path, crs={'init':'epsg:4686'})

In [ ]:
avaluo = avaluo[['CP_TERR_AR', 'GRUPOP_TER', 'AVALUO_COM', 'AVALUO_CAT', 'geometry']]

In [ ]:
avaluo = avaluo[
    (avaluo['AVALUO_CAT']<avaluo['AVALUO_CAT'].quantile(0.95)) &
    (avaluo['AVALUO_CAT']>avaluo['AVALUO_CAT'].quantile(0.05)) &
    (avaluo['AVALUO_COM']<avaluo['AVALUO_COM'].quantile(0.95)) &
    (avaluo['AVALUO_COM']>avaluo['AVALUO_COM'].quantile(0.05))
]

In [ ]:
pd.concat(
    [avaluo[['AVALUO_CAT', 'AVALUO_COM']].quantile(np.linspace(0,1,21)).astype('int64'),
    casas[['y']].quantile(np.linspace(0,1,21)).astype('int64')],
    axis=1).boxplot()

In [ ]:
pd.concat(
    [(avaluo['AVALUO_CAT'] + avaluo['AVALUO_COM'] + 650000).quantile(np.linspace(0,1,21)).astype('int64'),
    casas[['y']].quantile(np.linspace(0,1,21)).astype('int64')],
    axis=1).boxplot()

In [ ]:
interseccion1 = geopandas.sjoin(left_df=casas, right_df=estratificacion, how='left', op='intersects').drop(columns=['index_right'])

In [ ]:
sns.boxplot(y='y', x='ESTRATO', data=interseccion1[interseccion1['ESTRATO'].notna()])
plt.show()

In [ ]:
interseccion2 = geopandas.sjoin(left_df=interseccion1, right_df=avaluo, how='left', op='intersects').drop(columns=['index_right'])

In [ ]:
sns.boxplot(y='y', x='CP_TERR_AR', data=interseccion2[['y','CP_TERR_AR']].fillna('No define'))
plt.show()

In [ ]:
sns.boxplot(x='y', y='GRUPOP_TER', data=interseccion2[['y','GRUPOP_TER']].fillna('No define'), orient='h')
plt.show()

In [ ]:
interseccion3 = geopandas.sjoin(left_df=interseccion2, right_df=localidades, how='left', op='intersects').drop(columns=['index_right'])

In [ ]:
sns.boxplot(x='y', y='LocNombre', data=interseccion3.dropna())
plt.show()

In [ ]:
troncales_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\transmilenio'
troncales = geopandas.read_file(troncales_path, crs={'init':'epsg:4686'})

In [ ]:
colegios_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\colegios.csv'
colegios = pd.read_csv(colegios_path, encoding='latin1', sep=';')
colegios.X = colegios.X.str.replace(',','.').astype(float)

In [ ]:
colegios = geopandas.GeoDataFrame(
    colegios, geometry=geopandas.points_from_xy(colegios.X, colegios.Y), crs={'init': 'epsg:4326'})

In [ ]:
colegios['geometry'] = colegios.buffer(0.01)

In [ ]:
columna_colegios = ['COLEGIO ELISA BORRERO DE PASTRANA', 'CENTRO LESTONNAC - COMPAÑIA DE MARIA']
colegios_sub = colegios[colegios.NOMBRE_ESTABLECIMIENTO_EDUCATIVO.isin(columna_colegios)]

In [ ]:
bases1 = localidades.plot(color='white', edgecolor='black', figsize=(10, 10), zorder=1)
bases2 = troncales.plot(ax=bases1, figsize=(10, 10), zorder=2, color='blue', markersize=.1)
bases3 = colegios_sub.head().buffer(0.01).plot(ax=bases2, figsize=(10, 10), zorder=3, color='red',alpha=0.5)
bases4 = colegios_sub.head().plot(ax=bases3, figsize=(10, 10), zorder=3, color='red',markersize=.1)
bases5 = interseccion3[interseccion3['LocNombre']=='ENGATIVA'].sample(50, random_state=42).plot(ax=bases3, figsize=(10, 10), zorder=3, color='orange',markersize=.1)

In [ ]:
cant_colegios = geopandas.sjoin(
    left_df=casas[['id', 'geometry']], 
    right_df=colegios[['NOMBRE_ESTABLECIMIENTO_EDUCATIVO', 'geometry']], 
    how='left', 
    op='intersects').drop(columns=['index_right']).\
        groupby('id', as_index=False, dropna=False).\
        agg(**{'cantidad de colegios cerca': pd.NamedAgg('NOMBRE_ESTABLECIMIENTO_EDUCATIVO', 'count')})

In [ ]:
interseccion4 = pd.merge(interseccion3, cant_colegios, how='inner', on='id')

In [ ]:
sns.scatterplot(x='cantidad de colegios cerca', y='y', data=interseccion4[['y','cantidad de colegios cerca']].dropna())
plt.show()

In [ ]:
condlist = [
    (interseccion4['cantidad de colegios cerca']==0),
    (interseccion4['cantidad de colegios cerca']>=1) & (interseccion4['cantidad de colegios cerca']<11),
    (interseccion4['cantidad de colegios cerca']>=11) & (interseccion4['cantidad de colegios cerca']<21),
    (interseccion4['cantidad de colegios cerca']>=21) & (interseccion4['cantidad de colegios cerca']<31),
    (interseccion4['cantidad de colegios cerca']>=31) & (interseccion4['cantidad de colegios cerca']<41),
    (interseccion4['cantidad de colegios cerca']>=41)]
choicelist = ['0', '1-10', '11-20', '21-30', '31-40', '41+']
interseccion4['Agrupación de colegios cerca'] = np.select(condlist=condlist, choicelist=choicelist)

In [ ]:
sns.boxplot(
    x = 'cantidad de colegios cerca modificado', 
    y='y', 
    data=interseccion4[['cantidad de colegios cerca modificado', 'y']].dropna(),
    order=choicelist)
plt.show()

In [ ]:
g = sns.PairGrid(interseccion4[['y', 'cantidad de colegios cerca', 'Agrupación de colegios cerca']].sample(5000), hue='Agrupación de colegios cerca', diag_sharey=False)
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot)
plt.show()

In [ ]:
caps_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\CAPS'
caps = geopandas.read_file(caps_path, crs={'init':'epsg:4686'})

In [ ]:
caps = caps[['ID', 'geometry']]

In [ ]:
caps['geometry'] = caps.buffer(0.01)

In [ ]:
bases1 = localidades.plot(color='white', edgecolor='black', figsize=(10, 10), zorder=1)
bases2 = caps.plot(ax=bases1, figsize=(10, 10), zorder=3, color='blue', markersize=.1)
bases3 = caps.buffer(0.01).plot(ax=bases2, figsize=(10, 10), zorder=4, color='blue', alpha=0.5)
bases4 = interseccion4.sample(1000, random_state=42).plot(ax=bases3, figsize=(10, 10), zorder=5, color='orange', markersize=.1)

In [ ]:
cant_caps = geopandas.sjoin(
    left_df=casas[['id', 'geometry']], 
    right_df=caps[['ID', 'geometry']], 
    how='left', 
    op='intersects').drop(columns=['index_right']).\
        groupby('id', as_index=False, dropna=False).\
        agg(**{'cantidad de caps cerca': pd.NamedAgg('ID', 'count')})

In [ ]:
interseccion5 = pd.merge(interseccion4, cant_caps, how='inner', on='id')

In [ ]:
sns.violinplot(x='cantidad de caps cerca', y='y', data=interseccion5[['y','cantidad de caps cerca']].dropna())

In [ ]:
interseccion5.info()

In [ ]:
sns.heatmap(interseccion5[['y', 'area', 'valorventa','ESTRATO', 'habitaciones', 'CP_TERR_AR', 'AVALUO_COM', 'AVALUO_CAT','LocNombre','cantidad de caps cerca', 'cantidad de colegios cerca']].dropna().corr(), cmap='Greys')
plt.show()

In [ ]:
 interseccionn = interseccion5

In [ ]:
interseccionn = interseccionn[interseccionn['LocNombre'].notna()]

## Estrato

In [ ]:
interseccionn['ESTRATO'] = np.where(
    interseccionn['ESTRATO'].isna() | interseccionn['ESTRATO']==0,
    np.NaN,
    interseccionn['ESTRATO']
)

In [ ]:
train = interseccionn.loc[interseccionn['ESTRATO'].notna(),['ESTRATO', 'latitud', 'longitud']]
X = train[['latitud', 'longitud']].values
y = train[['ESTRATO']].values
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
neigh = KNeighborsClassifier(n_neighbors=3, metric = 'euclidean')
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)
print("R^2: {}".format(neigh.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))
print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))

In [ ]:
valores = interseccionn.loc[interseccionn['ESTRATO'].isnull(),['latitud', 'longitud']].values
interseccionn.loc[interseccionn['ESTRATO'].isnull(),'ESTRATO'] = neigh.predict(valores)

## AVALUO_COM

In [ ]:
train = interseccionn.loc[interseccionn['AVALUO_COM'].notna(),['AVALUO_COM', 'latitud', 'longitud']]
X = train[['latitud', 'longitud']].values
y = train[['AVALUO_COM']].values
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
neigh = DecisionTreeRegressor()
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)
print("R^2: {}".format(neigh.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))

In [ ]:
valores = interseccionn.loc[interseccionn['AVALUO_COM'].isnull(),['latitud', 'longitud']].values
interseccionn.loc[interseccionn['AVALUO_COM'].isnull(),'AVALUO_COM'] = neigh.predict(valores)

## AVALUO_CAT

In [ ]:
train = interseccionn.loc[interseccionn['AVALUO_CAT'].notna(),['AVALUO_CAT', 'latitud', 'longitud']]
X = train[['latitud', 'longitud']].values
y = train[['AVALUO_CAT']].values
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
neigh = DecisionTreeRegressor()#algorithm='brute', n_neighbors=5,  leaf_size=1, p=2, weights='distance', metric='chebyshev', n_jobs=-1)
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)
print("R^2: {}".format(neigh.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))

In [ ]:
valores = interseccionn.loc[interseccionn['AVALUO_CAT'].isnull(),['latitud', 'longitud']].values
interseccionn.loc[interseccionn['AVALUO_CAT'].isnull(),'AVALUO_CAT'] = neigh.predict(valores)

## GRUPOP_TER

In [ ]:
interseccionn[interseccionn['GRUPOP_TER'].notna()]['GRUPOP_TER'].unique()

In [ ]:
interseccionn[interseccionn['GRUPOP_TER'].notna()]['GRUPOP_TER'].unique()

In [ ]:
indices = {}
for i, grupo in enumerate(interseccionn[interseccionn['GRUPOP_TER'].notna()]['GRUPOP_TER'].unique()):
    indices[i] = grupo
    interseccionn.loc[interseccionn['GRUPOP_TER']==grupo, ['GRUPOP_TER']] = i

In [ ]:
train = interseccionn.loc[interseccionn['GRUPOP_TER'].notna(),['GRUPOP_TER', 'latitud', 'longitud']]
X = train[['latitud', 'longitud']].values
y = train[['GRUPOP_TER']].astype(int).values
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
neigh = KNeighborsClassifier()#algorithm='brute', n_neighbors=5,  leaf_size=1, p=2, weights='distance', metric='chebyshev', n_jobs=-1)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
print("R^2: {}".format(neigh.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))
print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))

In [ ]:
valores = interseccionn.loc[interseccionn['GRUPOP_TER'].isnull(),['latitud', 'longitud']].values
interseccionn.loc[interseccionn['GRUPOP_TER'].isnull(),'GRUPOP_TER'] = neigh.predict(valores)

## CP_TERR_AR

In [ ]:
indices2 = {}
for i, grupo in enumerate(interseccionn[interseccionn['CP_TERR_AR'].notna()]['CP_TERR_AR'].unique()):
    indices2[i] = grupo
    interseccionn.loc[interseccionn['CP_TERR_AR']==grupo, ['CP_TERR_AR']] = i

In [ ]:
train = interseccionn.loc[interseccionn['CP_TERR_AR'].notna(),['CP_TERR_AR', 'latitud', 'longitud']]
X = train[['latitud', 'longitud']].values
y = train[['CP_TERR_AR']].astype(int).values
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
neigh = KNeighborsClassifier()
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
print("R^2: {}".format(neigh.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))
print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))

In [ ]:
valores = interseccionn.loc[interseccionn['CP_TERR_AR'].isnull(),['latitud', 'longitud']].values
interseccionn.loc[interseccionn['CP_TERR_AR'].isnull(),'CP_TERR_AR'] = neigh.predict(valores)

In [ ]:
for i, grupo in indices.items():
    interseccionn.loc[interseccionn['GRUPOP_TER']==i, ['GRUPOP_TER']] = grupo

In [ ]:
for i, grupo in indices2.items():
    interseccionn.loc[interseccionn['CP_TERR_AR']==i, ['CP_TERR_AR']] = grupo

In [ ]:
sns.boxplot(y='GRUPOP_TER', x='y', data=interseccionn[['y','GRUPOP_TER']].dropna())
plt.show()

In [ ]:
sns.boxplot(x='CP_TERR_AR', y='y', data=interseccionn[['y','CP_TERR_AR']].dropna())
plt.show()

In [ ]:
interseccionn = interseccionn.sort_values(["ESTRATO", "id"])

In [ ]:
probabilities = []
for estrato in interseccionn['ESTRATO'].sort_values().unique():
    with localconverter(ro.default_converter + pandas2ri.converter):
        r_from_pd_df = ro.conversion.py2rpy(interseccionn[interseccionn['ESTRATO']==estrato][["id","y"]])
    model = gamlss.gamlss(formula = rpackages.reval("y~1"), data=r_from_pd_df, family = "GA")
    probs = gamlss_dist.pGA(
        ro.IntVector(interseccionn[interseccionn['ESTRATO']==estrato]["y"]), 
        mu=base.exp(model[24]), 
        sigma=base.exp(model[41]))
    probabilities.extend(probs)

In [ ]:
probabilities = np.array(probabilities).reshape(1,-1)

In [ ]:
interseccionn['y_estimado'] = (
    interseccionn['y'].values*(100-MinMaxScaler(feature_range=(2,4)).fit(probabilities).transform(probabilities).reshape(-1))
)/100

In [ ]:
interseccionn = interseccionn.set_index('id')

In [ ]:
interseccionn.info()

In [ ]:
eliminar_columnas = [
    'deposito', 'estrato', 'garajes', 'piso', 'tiempodeconstruido', 'vigilancia',
    'vista', 'zonalavanderia'
]
interseccionn = interseccionn.drop(columns=eliminar_columnas)

In [ ]:
interseccionn[interseccionn.columns].corr()['y_estimado']

In [ ]:
columnas_dummy = [
    'banos modificado', 'garajes modificado', 'piso modificado', 'vista modificado',
    'vista modificado', 'vigilancia modificado', 'tiempodeconstruido modificado', 
    'CP_TERR_AR', 'LocNombre', 'cantidad de colegios cerca modificado'
    ,'GRUPOP_TER'
     ]
# 'LocNombre', 'V_REF', ,
data1 = pd.get_dummies(
    interseccionn.drop(columns=['geometry',  'latitud', 'longitud', 'banos', 'prob', 'Agrupación de colegios cerca']),
    drop_first=True, 
    columns=columnas_dummy)

In [ ]:
data_test = data1[data1['data']=='test'].drop(columns=['data'])
data1 = data1[data1['data']=='train'].drop(columns=['data'])

In [ ]:
X = data1.drop(columns=['y_estimado']).values
y = data1[['y_estimado']].values

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(data_test.shape)

## Modelo RandomForest

In [ ]:
mod2 = RandomForestRegressor(random_state=0, n_jobs=-1).fit(X_train, y_train.ravel())

In [ ]:
print(mean_absolute_percentage_error(y_train, mod2.predict(X_train)))
print(mean_absolute_percentage_error(y_test, mod2.predict(X_test)))
print(mean_absolute_percentage_error(
    data_test[['y_estimado']].values,
    mod2.predict(data_test.drop(columns=['y_estimado']).values)))

In [ ]:
print(mean_absolute_error(
    data_test[['y_estimado']].values,
    mod2.predict(data_test.drop(columns=['y_estimado']).values)))

In [ ]:
# resultados = mod2.predict(data_test.drop(columns=['y']).values)
# resultados_df = pd.DataFrame(resultados, index=data_test.index, columns=['valormt2_predicted'])\
#     .reset_index(drop=False).rename(columns={'index':'id'})
# resultados_df.to_csv(r'C:\Users\William\Desktop\el algoritmo es correcto\resultado5.csv', index=False)

## Modelo XGBOOST

In [ ]:
data = data1.drop(columns=['y_estimado']).values
label = data1['y_estimado'].values
dtrain = xgb.DMatrix(data, label=label)

In [ ]:
data = data_test.drop(columns=['y_estimado']).values
label = data_test['y_estimado'].values
dtest2 = xgb.DMatrix(data, label=label)

In [ ]:
squared_error = {
    # 'booster': 'dart',
    'lambda': 1,#
    'alpha': 1,#
    # 'updater': 'shotgun',#
    'feature_selector': 'greedy', # Variaciones
    # 'top_k': 0, #
    'objective': 'reg:squarederror', #
    'base_score': 1,
    'eval_metric': 'mape',
    'tree_method': 'exact',
    'seed': np.random.seed(1994)
    }

In [ ]:
results: Dict[str, Dict[str, List[float]]] = {}

In [ ]:
kBoostRound = 500
bst = xgb.train(
    squared_error,
    dtrain=dtrain,
    num_boost_round=kBoostRound,
    evals=[(dtrain, 'dtrain'), (dtest2, 'dtest')],
    evals_result=results)

In [ ]:
print(mean_absolute_percentage_error(
    data_test['y_estimado'].values,
    bst.predict(dtest2, ntree_limit=bst.best_ntree_limit)))

In [ ]:
mean_absolute_error(
    data_test['y_estimado'].values,
    bst.predict(dtest2, ntree_limit=bst.best_ntree_limit)
)

In [ ]:
bst.predict(dtest2, ntree_limit=bst.best_ntree_limit)

In [ ]:
# resultados = bst.predict(dtest2, ntree_limit=bst.best_ntree_limit)
# resultados_df = pd.DataFrame(resultados, index=data_test.index, columns=['valormt2_predicted'])\
#     .reset_index(drop=False).rename(columns={'index':'id'})
# resultados_df.to_csv(r'C:\Users\William\Desktop\el algoritmo es correcto\resultado11.csv', index=False)